In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Load dataset
file_path = "/content/10-Java_AST_in_.xlsx"
df = pd.read_excel(file_path, sheet_name="in")

# Handle missing values (fill with 0 or use df.dropna())
df = df.fillna(0)

# Ensure all data is numeric
df = df.apply(pd.to_numeric, errors='coerce').fillna(0)

# Define features and target variable
X = df.drop(columns=["Final_Marks", "error_count"], errors='ignore')  # Features
y = df["Final_Marks"]  # Target variable

# Split data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models with fixed random_state
models = {
    "SVR": SVR(),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "RandomForest": RandomForestRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42),
    "AdaBoost": AdaBoostRegressor(random_state=42),
    "MLPRegressor": MLPRegressor(max_iter=500)
}

# Dictionary to store results
results = []

# Train and evaluate models
for name, model in models.items():
    print(f"Training {name}...")  # Show progress
    model.fit(X_train, y_train)  # Train model
    y_train_pred = model.predict(X_train)  # Predictions on training set
    y_test_pred = model.predict(X_test)  # Predictions on test set

    # Compute metrics
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    train_mae = mean_absolute_error(y_train, y_train_pred)
    test_mae = mean_absolute_error(y_test, y_test_pred)
    train_rmse = np.sqrt(train_mse)
    test_rmse = np.sqrt(test_mse)

    # Store results
    results.append([name, train_r2, test_r2, train_mse, test_mse, train_mae, test_mae, train_rmse, test_rmse])

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=["Model", "Train R²", "Test R²", "Train MSE", "Test MSE",
                                            "Train MAE", "Test MAE", "Train RMSE", "Test RMSE"])

# Display results
print("\nRegression Model Performance:\n")
print(results_df)


from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint

# Define parameter grid for Random Forest
param_dist = {
    'n_estimators': randint(50, 300),
    'max_depth': [None] + list(range(5, 30)),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 5),
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize base model
rf = RandomForestRegressor(random_state=42)

# RandomizedSearchCV setup
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,
    scoring='r2',
    cv=5,
    verbose=2,
    random_state=42,
    n_jobs=-1
)

# Fit the model
random_search.fit(X_train, y_train)

# Best parameters & performance
print("Best Parameters:", random_search.best_params_)
print("Best CV Score (R²):", random_search.best_score_)

# Evaluate on test set
best_rf = random_search.best_estimator_
y_test_pred = best_rf.predict(X_test)
y_train_pred = best_rf.predict(X_train)

# Final performance
print("\nTrain R²:", r2_score(y_train, y_train_pred))
print("Test R²:", r2_score(y_test, y_test_pred))
print("Test RMSE:", np.sqrt(mean_squared_error(y_test, y_test_pred)))


Training SVR...
Training DecisionTree...
Training RandomForest...
Training XGBoost...
Training AdaBoost...
Training MLPRegressor...

Regression Model Performance:

          Model  Train R²   Test R²  Train MSE  Test MSE  Train MAE  Test MAE  \
0           SVR  0.341206  0.368964   3.844483  3.784437   1.554140  1.519559   
1  DecisionTree  0.865723  0.174229   0.783591  4.952299   0.426602  1.618244   
2  RandomForest  0.807159  0.464476   1.125352  3.211637   0.781878  1.413633   
3       XGBoost  0.865723  0.415666   0.783594  3.504359   0.427584  1.433336   
4      AdaBoost  0.459956  0.370764   3.151500  3.773645   1.511961  1.540480   
5  MLPRegressor  0.769611  0.483492   1.344469  3.097593   0.834679  1.359127   

   Train RMSE  Test RMSE  
0    1.960735   1.945363  
1    0.885207   2.225376  
2    1.060826   1.792104  
3    0.885208   1.871993  
4    1.775246   1.942587  
5    1.159512   1.759998  
Fitting 5 folds for each of 50 candidates, totalling 250 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
85 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
18 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1382, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 436, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.11/dist-packages/sklearn/utils/

Best Parameters: {'max_depth': 27, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 3, 'n_estimators': 241}
Best CV Score (R²): 0.3708804720240887

Train R²: 0.7946586928107376
Test R²: 0.4709567501203893
Test RMSE: 1.7812265411306982
